In [8]:
import numpy as np
import os
from os.path import expanduser
home_dir = expanduser("~")
module_path = home_dir + '/code/modules/'
import sys
sys.path.append(module_path)
import pandas as pd
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from numba import jit
import time, datetime
%load_ext autoreload
%autoreload 1
%aimport environmental_density
from environmental_density import get_density_periodic
np.random.seed(999)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
data_dict = {'X_pos': 0, 'Y_pos': 1, 'Z_pos': 2, 'X_vel': 3, 'Y_vel': 4, 'Z_vel': 5, 'Halo_mass': 6, 
             'Stellar_mass': 7, 'SFR': 8, 'Intra_cluster_mass': 9, 'Halo_mass_peak': 10, 'Stellar_mass_obs': 11, 
             'SFR_obs': 12, 'Halo_radius': 13, 'Concentration': 14, 'Halo_spin': 15, 'Scale_peak_mass': 16, 
             'Scale_half_mass': 17, 'Scale_last_MajM': 18, 'Type': 19}

In [12]:
origin_galcat_directory = '/home/magnus/data/galcats_nonzero_sfr_no_density/'
destination_galcat_directory = '/home/magnus/data/galcats_nonzero_sfr_with_density/'
redshifts = [0, 1, 2, 5, 10, 20, 30, 40, 60, 80]
with open('progress.txt', 'w+') as f:
    for redshift in redshifts:
        f.write('{}    Working on redshift Z{:02d}'.format(datetime.datetime.now().strftime("%H:%M:%S"), redshift))
        f.flush()
        
        file_name = 'galaxies.Z{:02d}.h5'.format(redshift)
        galfile = pd.read_hdf(origin_galcat_directory + file_name)
        galaxies = galfile.as_matrix()
        gal_header = galfile.keys().tolist()
        print(gal_header)

        ### Remove data points with halo mass below 10.5
        galaxies = galaxies[galaxies[:,6] > 10.5, :]

        coordinates = galaxies[:, :3]
        halo_masses = np.power(10, galaxies[:, data_dict['Halo_mass']])
        nr_points = np.shape(coordinates)[0]

        nr_neighbours_wanted = 10
        box_sides = np.array([200, 200, 200])

        neigh_densities = get_density_periodic(coordinates, halo_masses, nr_neighbours_wanted, 
                                                box_sides, nr_points, verbatim=False, progress_file=False)
        neigh_densities = np.log10(neigh_densities)

        pd_dataframe = pd.DataFrame(data=galaxies, columns=gal_header)
        pd_dataframe['Environmental_density'] = neigh_densities

        pd_dataframe.to_hdf(destination_galcat_directory + file_name, 'w')

['X_pos', 'Y_pos', 'Z_pos', 'X_vel', 'Y_vel', 'Z_vel', 'Halo_mass', 'Stellar_mass', 'SFR', 'Intra_cluster_mass', 'Halo_mass_peak', 'Stellar_mass_obs', 'SFR_obs', 'Halo_radius', 'Concentration', 'Halo_spin', 'Scale_peak_mass', 'Scale_half_mass', 'Scale_last_MajM', 'Type']
['X_pos', 'Y_pos', 'Z_pos', 'X_vel', 'Y_vel', 'Z_vel', 'Halo_mass', 'Stellar_mass', 'SFR', 'Intra_cluster_mass', 'Halo_mass_peak', 'Stellar_mass_obs', 'SFR_obs', 'Halo_radius', 'Concentration', 'Halo_spin', 'Scale_peak_mass', 'Scale_half_mass', 'Scale_last_MajM', 'Type']
['X_pos', 'Y_pos', 'Z_pos', 'X_vel', 'Y_vel', 'Z_vel', 'Halo_mass', 'Stellar_mass', 'SFR', 'Intra_cluster_mass', 'Halo_mass_peak', 'Stellar_mass_obs', 'SFR_obs', 'Halo_radius', 'Concentration', 'Halo_spin', 'Scale_peak_mass', 'Scale_half_mass', 'Scale_last_MajM', 'Type']
['X_pos', 'Y_pos', 'Z_pos', 'X_vel', 'Y_vel', 'Z_vel', 'Halo_mass', 'Stellar_mass', 'SFR', 'Intra_cluster_mass', 'Halo_mass_peak', 'Stellar_mass_obs', 'SFR_obs', 'Halo_radius', 'Concen

In [5]:
### Make sure that the edges are what they should be
print(np.min(galaxies[:, 0]), np.max(galaxies[:, 0]))
print(np.min(galaxies[:, 1]), np.max(galaxies[:, 1]))
print(np.min(galaxies[:, 2]), np.max(galaxies[:, 2]))

0.00088534754 199.9999
0.0 199.99922
0.00028036005 199.99968


In [ ]:
print(np.mean(neigh_densities))
print(np.sum(halo_masses)/200**3)

In [ ]:
print(neigh_indices[3])

## Plot the data points to make sure that the correct neighbors were chosen

In [ ]:
not_found_middle = True
not_found_corner = True
point_counter = 0
while not_found_middle or not_found_corner:
    
    if not_found_corner:
        
        is_in_corner = (coordinates[point_counter,0] < 10 and coordinates[point_counter,1] < 10 and 
                     coordinates[point_counter,2] < 10)
        if is_in_corner:
            not_found_corner = False
            corner_point = point_counter
            
    if not_found_middle:
        is_in_middle = (coordinates[point_counter,0] < 110 and coordinates[point_counter,1] < 110 and 
                        coordinates[point_counter,2] < 110 and coordinates[point_counter,0] > 90 and 
                        coordinates[point_counter,1] > 90 and coordinates[point_counter,2] > 90)
        if is_in_middle:
            not_found_middle = False
            middle_point = point_counter
    point_counter += 1

fig = plt.figure(5, figsize=(16,16))
ax = plt.subplot(221)

ax.plot(coordinates[:,0], coordinates[:,1], 'k.', markersize=1)
neigh_points = ax.plot(coordinates[neigh_indices[middle_point],0], 
                       coordinates[neigh_indices[middle_point],1], 'bo', markersize=3, label='neighbours')
mid_point = ax.plot(coordinates[middle_point,0], coordinates[middle_point,1], 'ro', markersize=7, label='center point')
plt.title('XY plane', fontsize=25)
plt.ylabel('Y', fontsize=20)
plt.xlabel('X', fontsize=20)
ax.legend(loc='upper center')

ax = plt.subplot(222)
ax.plot(coordinates[:,0], coordinates[:,2], 'k.', markersize=1)
neigh_points = ax.plot(coordinates[neigh_indices[middle_point],0], 
                       coordinates[neigh_indices[middle_point],2], 'bo', markersize=3, label='neighbours')
mid_point = ax.plot(coordinates[middle_point,0], coordinates[middle_point,2], 'ro', markersize=7, label='center point')
plt.title('XZ plane', fontsize=25)
plt.ylabel('Z', fontsize=20)
plt.xlabel('X', fontsize=20)
ax.legend(loc='upper center')

#plt.show()

#fig = plt.figure(5, figsize=(16,8))
ax = plt.subplot(223)

ax.plot(coordinates[:,0], coordinates[:,1], 'k.', markersize=1)
ax.plot(coordinates[neigh_indices[corner_point],0], 
                       coordinates[neigh_indices[corner_point],2], 'bo', markersize=3, label='neighbours')
ax.plot(coordinates[corner_point,0], coordinates[corner_point,1], 'ro', markersize=7, label='center point')
plt.title('XY plane', fontsize=25)
plt.ylabel('Y', fontsize=20)
plt.xlabel('X', fontsize=20)
ax.legend(loc='upper center')

ax = plt.subplot(224)
ax.plot(coordinates[:,0], coordinates[:,2], 'k.', markersize=1)
neigh_points = ax.plot(coordinates[neigh_indices[corner_point],0], 
                       coordinates[neigh_indices[corner_point],2], 'bo', markersize=3, label='neighbours')
ax.plot(coordinates[corner_point,0], coordinates[corner_point,2], 'ro', markersize=7, label='center point')
plt.title('XZ plane', fontsize=25)
plt.ylabel('Z', fontsize=20)
plt.xlabel('X', fontsize=20)
ax.legend(loc='upper center')

plt.show()

In [ ]:
### Save the figure
fig.savefig('neighbours_visualisation.png', bbox_inches = 'tight')

## Make a new pd dataset and save it

## Try loading the newly created galaxy catalogue

In [24]:
galfile = pd.read_hdf(destination_galcat_directory + 'galaxies.Z80.h5')
galaxies = galfile.as_matrix()
gal_header = galfile.keys().tolist()
print(gal_header)
print(np.shape(galaxies))

print('%.2f' % (np.mean(galaxies[:,20])))
print('%.2f' % (np.amax(galaxies[:,20])))
print('%.2f' % (np.amin(galaxies[:,20])))
#print(np.sum(galaxies[:,6]) / 200**3)

['X_pos', 'Y_pos', 'Z_pos', 'X_vel', 'Y_vel', 'Z_vel', 'Halo_mass', 'Stellar_mass', 'SFR', 'Intra_cluster_mass', 'Halo_mass_peak', 'Stellar_mass_obs', 'SFR_obs', 'Halo_radius', 'Concentration', 'Halo_spin', 'Scale_peak_mass', 'Scale_half_mass', 'Scale_last_MajM', 'Type', 'Environmental_density']
(5971, 21)
8.17
11.54
6.61


In [ ]:
galaxies[:,20] = np.log10(galaxies[:,20])

## Old functions

In [ ]:
@jit(nopython=True)
def nearest_k_neighbours_1(coordinates, nr_neighbours, box_sides, nr_points):
    indices_of_neighbors = np.zeros((coordinates.shape[0], nr_neighbours))
    for i in range(nr_points):

        euclidean_dists = np.sqrt(np.sum(np.minimum(np.abs(coordinates - coordinates[i,:]), box_sides - 
                                                    (np.abs(coordinates - coordinates[i,:])))**2, 1))
        indices_of_neighbors[i,:] = np.argsort(euclidean_dists)[:nr_neighbours]
    
    return indices_of_neighbors

In [ ]:
@jit(nopython=True)
def nearest_k_neighbours_2(coordinates, nr_neighbours, box_sides, nr_points):
    indices_of_neighbors = np.zeros((coordinates.shape[0], nr_neighbours))
#    weights_of_neighbors = np.zeros((coordinates.shape[0], nr_neighbours))
    for i in range(nr_points):
        norm_dist = np.abs(coordinates - coordinates[i,:])
        inv_dist = box_sides - norm_dist
        shortest_dists = np.minimum(norm_dist, inv_dist)
        squared_shortest_dists = shortest_dists**2
        summed_squared_shortest_dists = np.sum(squared_shortest_dists, 1)
        euclidean_dists = np.sqrt(summed_squared_shortest_dists)
        
        indices_of_neighbors[i,:] = np.argsort(euclidean_dists)[:nr_neighbours]
#        weights_of_neighbors[i,:]
    
    return indices_of_neighbors

In [ ]:
def nearest_k_neighbours_3(coordinates, nr_neighbours, box_sides, nr_points):
    indices_of_neighbors = np.zeros((coordinates.shape[0], nr_neighbours))
    for i in range(nr_points):

        euclidean_dists = np.sqrt(np.sum(np.minimum(np.abs(coordinates - coordinates[i,:]), box_sides - 
                                                    (np.abs(coordinates - coordinates[i,:])))**2, 1))
        indices_of_neighbors[i,:] = np.argpartition(euclidean_dists, nr_neighbours)[:nr_neighbours]
    
    return indices_of_neighbors

In [ ]:
@jit
def get_density_periodic_jit(coordinates, weights, nr_neighbours, box_sides, nr_points):
    indices_of_neighbors = np.zeros((coordinates.shape[0], nr_neighbours), dtype=np.int8)
    masses_of_neighbors = np.zeros((coordinates.shape[0], nr_neighbours))
    neigh_densities = np.zeros(nr_points)
    for i in range(nr_points):
        norm_dist = np.abs(coordinates - coordinates[i,:])
        inv_dist = box_sides - norm_dist
        shortest_dists = np.minimum(norm_dist, inv_dist)
        squared_shortest_dists = shortest_dists**2
        summed_squared_shortest_dists = np.sum(squared_shortest_dists, 1)
        euclidean_dists = np.sqrt(summed_squared_shortest_dists)
        
        indices_of_k_closest_neighbours = np.argpartition(euclidean_dists, nr_neighbours)[:nr_neighbours]
        indices_of_neighbors[i,:] = indices_of_k_closest_neighbours
        masses_of_neighbors[i,:] = weights[indices_of_k_closest_neighbours]
        
        sphere_radius = np.max(euclidean_dists[indices_of_k_closest_neighbours])
        print('sphere radius: ', sphere_radius)
        sphere_volume = (4/3) * math.pi * sphere_radius**3
#        print('sphere volume: ', sphere_volume)
        density =  np.sum(masses_of_neighbors[i,:]) / sphere_volume
#        print('density: ', density)
        neigh_densities[i] = density
    
    return indices_of_neighbors, masses_of_neighbors, neigh_densities
                

In [ ]:
#@jit
def get_density_periodic_OLD(coordinates, weights, nr_neighbours, box_sides, nr_points, verbatim=False):
    
    with open('results.txt', 'w+') as f:

        neigh_densities = np.zeros(nr_points)

        start = time.time()
        for i in range(nr_points):

            if int(i/1000) == i/1000 and i > 0:
                end = time.time()
                elapsed_time = (end-start)/60
                time_remaining = elapsed_time / i * (nr_points - i)
                if verbatim:
                    print('%s      Time elapsed: %.2fmin. Time remaining: %.2fmin.' % (datetime.datetime.now().strftime("%H:%M:%S"),
                                                                            elapsed_time, time_remaining))
                f.write('%s      Time elapsed: %.2fmin. Time remaining: %.2fmin.\n' % (datetime.datetime.now().strftime("%H:%M:%S"),
                                                                        elapsed_time, time_remaining))
                f.flush()
            norm_dist = np.abs(coordinates - coordinates[i,:])
            inv_dist = box_sides - norm_dist
            shortest_dists = np.minimum(norm_dist, inv_dist)
            squared_shortest_dists = shortest_dists**2
            summed_squared_shortest_dists = np.sum(squared_shortest_dists, 1)
            euclidean_dists = np.sqrt(summed_squared_shortest_dists)

            indices_of_k_closest_neighbours = np.argpartition(euclidean_dists, nr_neighbours)[:nr_neighbours]
            masses_of_k_closest_neighbours = weights[indices_of_k_closest_neighbours]

            sphere_radius = np.amax(euclidean_dists[indices_of_k_closest_neighbours])

            if sphere_radius > box_sides[0]/2:
                print('WARNING! Sphere with radius larger than %d mega parsecs created.' % (box_sides[0]/2))
                f.write('WARNING! Sphere with radius larger than %d mega parsecs created.\n' % (box_sides[0]/2))
                f.flush()
            sphere_volume = (4/3) * math.pi * sphere_radius**3
            density =  np.sum(masses_of_k_closest_neighbours) / sphere_volume
            neigh_densities[i] = density
            
        elapsed_time = (end-start)/60
        time_remaining = elapsed_time / i * (nr_points - i)
        f.write('%s      Script finished. Elapsed time: %.2fmin.' % (datetime.datetime.now().strftime("%H:%M:%S"), elapsed_time))
    
    return neigh_densities